In [ ]:
import os

In [ ]:
BUCKET = 'gs://edml/data/taxi-trips'

In [ ]:
os.environ['BUCKET'] = BUCKET

## Local training / Python

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
echo $PYTHONPATH

In [ ]:
%%bash
echo "bucket=${BUCKET}"
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
python3 -m trainer.task \
  --bucket=${BUCKET} \
  --output-dir=model_trained_test \
  --job-dir=./tmp \
  --train-examples=500 \
  --nembeds 10 \
  --nnsize 10 5 \
  --eval-steps=2

In [ ]:
%%bash
gsutil cp ./model_trained_test/export/exporter/1581540201/saved_model.pb gs://edml/tmp

In [ ]:
from google.datalab.ml import TensorBoard

In [ ]:
TensorBoard().start('./model_trained_test')
TensorBoard().list()

In [ ]:
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print('Stopped TensorBoard with pid {}'.format(pid))

In [ ]:
%%bash
export MODEL_DIR=tmp_tensorboard
tensorboard --logdir=$MODEL_DIR --port=6006

## Local prediction

In [ ]:
%%writefile inputs.json
{"uuid": "b1", "dayofweek": 6, "hourofday": 7, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Newark Airport", "passenger_count": 1}
{"uuid": "g1", "dayofweek": 3, "hourofday": 23, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Times Sq/Theatre District", "passenger_count": 1}

In [ ]:
%%bash
MODEL_LOCATION=$(ls -d $(pwd)/model_trained_test/export/exporter/* | tail -1)
echo $MODEL_LOCATION
gcloud ai-platform local predict --model-dir=$MODEL_LOCATION --json-instances=inputs.json

## Local training / Gcloud

In [ ]:
%%bash
gcloud ai-platform --help

In [ ]:
%%bash
echo $PATH

In [ ]:
%%bash
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer:/usr/bin/python
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=~/event-driven-ml/edml-trainer \
   --job-dir=./tmp \
   -- \
   --train-examples=500 \
   --output-dir=model_trained_test \
   --eval-steps=1